In [2]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [95]:
import pandas as pd
import numpy as np

from pydataset import data
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean, round
from pyspark.sql.functions import lit
from pyspark.sql.functions import when
from pyspark.sql.functions import col, expr

In [31]:
print(spark)
print(spark.version)

3.0.1


In [169]:
spark

# Exercises

### 1. Create a spark data frame that contains your favorite programming languages.

- The name of the column should be `language`
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [3]:
lang_df = pd.DataFrame({"language": ['python', 'spark', 'sql', 'swift', 'javascript']})

In [4]:
lang_df

,language
0,python
1,spark
2,sql
3,swift
4,javascript


In [5]:
sp_df = spark.createDataFrame(lang_df)
sp_df

DataFrame[language: string]

In [6]:
sp_df.printSchema()

root
 |-- language: string (nullable = true)



In [7]:
#printing the shape of the spark df:
print(sp_df.count(), len(sp_df.columns))

5 1


In [8]:
sp_df.show(5)

+----------+
|  language|
+----------+
|    python|
|     spark|
|       sql|
|     swift|
|javascript|
+----------+



### 2 Load the mpg dataset as a spark dataframe.

a. Create 1 column of output that contains a message like the one below:

`The 1999 audi a4 has a 4 cylinder engine. For each vehicle.`

b. Transform the trans column so that it only contains either manual or auto.

In [9]:
from pyspark.sql.functions import round, concat, sum, min, max, count, avg, mean
from pyspark.sql.functions import lit
from pyspark.sql.functions import when

In [20]:
# importing the data:
mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [12]:
from pyspark.sql.functions import regexp_extract, regexp_replace

In [13]:
# Use lit

mpg.select(concat(lit("The "), mpg.year, lit(" "), mpg.manufacturer, lit(" "), mpg.model, lit(" has a "), mpg.cyl, lit(" engine")).alias("cylinders")).show(1)

+--------------------+
|           cylinders|
+--------------------+
|The 1999 audi a4 ...|
+--------------------+
only showing top 1 row



In [14]:
mpg.select(mpg.trans, when(mpg.trans.contains("auto"), "auto").otherwise("manual").alias("Trans")).show(5)

+----------+------+
|     trans| Trans|
+----------+------+
|  auto(l5)|  auto|
|manual(m5)|manual|
|manual(m6)|manual|
|  auto(av)|  auto|
|  auto(l5)|  auto|
+----------+------+
only showing top 5 rows



In [ ]:
# Testing out how to use the regex for this problem. Although I think I'll just end up using basic python usage:



### 3. Load the tips dataset as a spark dataframe.

- What percentage of observations are smokers?
- Create a column that contains the tip percentage
- Calculate the average tip percentage for each combination of sex and smoker.

In [4]:
tips = spark.createDataFrame(data("tips"))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [26]:
tips.count()

244

In [32]:
# So this is the count of the smoker column, just need to make it %
tips.groupBy('smoker').agg(count(tips.smoker)).show()

+------+-------------+
|smoker|count(smoker)|
+------+-------------+
|    No|          151|
|   Yes|           93|
+------+-------------+



In [33]:
# % of observations that are smokers:

tips.groupBy('smoker').agg(round(count(tips.smoker)/ tips.count(),2)).show()

+------+-------------------------------+
|smoker|round((count(smoker) / 244), 2)|
+------+-------------------------------+
|    No|                           0.62|
|   Yes|                           0.38|
+------+-------------------------------+



#### B. Create a column that contains the tip percentage

In [41]:
# This is accomplished by dividing the tip / total_bill by row...

tips.select(tips.total_bill, tips.tip).show(5)

+----------+----+
|total_bill| tip|
+----------+----+
|     16.99|1.01|
|     10.34|1.66|
|     21.01| 3.5|
|     23.68|3.31|
|     24.59|3.61|
+----------+----+
only showing top 5 rows



In [109]:
# Now to divide so it's a percentage:

tips.select(tips.total_bill, tips.tip, round(tips.total_bill / tips.tip, 2).alias('tip_percentage')).show(5)

+----------+----+--------------+
|total_bill| tip|tip_percentage|
+----------+----+--------------+
|     16.99|1.01|         16.82|
|     10.34|1.66|          6.23|
|     21.01| 3.5|           6.0|
|     23.68|3.31|          7.15|
|     24.59|3.61|          6.81|
+----------+----+--------------+
only showing top 5 rows



#### C. Calculate the average tip percentage for each combination of sex and smoker.

- First need to aggregate so that I have 4 categories that look like this:

Female   smoker

male     smoker

female   non-smoker

male    smoker

And in that way I can create something that then does the math for each of those columns... in other words I'll need to classify each row by aggregation, and then divide the tip by the total bill for each of those groups.

In [81]:
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [84]:
tips.groupBy('smoker', 'sex', 'total_bill', 'tip').agg(round(tips.tip / tips.total_bill, 2).alias("tip_percentage")).show(5)

+------+------+----------+----+--------------+
|smoker|   sex|total_bill| tip|tip_percentage|
+------+------+----------+----+--------------+
|    No|  Male|      21.5| 3.5|          0.16|
|   Yes|Female|     43.11| 5.0|          0.12|
|    No|Female|     16.93|3.07|          0.18|
|   Yes|  Male|     15.48|2.02|          0.13|
|    No|Female|     22.75|3.25|          0.14|
+------+------+----------+----+--------------+
only showing top 5 rows



In [110]:
# Ok, so the above isn't what I wanted. What I need to do is create a new column, then groupby, then agg by the average of that new tip_percentage column.

In [100]:
tip_percentage = tips.withColumn("tip_percentage", (round(col('tip')/col('total_bill'), 2)))

In [101]:
tip_percentage.show()

+----------+----+------+------+---+------+----+--------------+
|total_bill| tip|   sex|smoker|day|  time|size|tip_percentage|
+----------+----+------+------+---+------+----+--------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|          0.06|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|          0.16|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|          0.17|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|          0.14|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|          0.15|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|          0.19|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|          0.23|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|          0.12|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|          0.13|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|          0.22|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|          0.17|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|          0.14|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|        

In [108]:
# Now putting it all together:
tips.groupBy('smoker', 'sex').agg(round(avg(tips.tip_percentage), 2).alias("tip_percentages")).show()

+------+------+---------------+
|smoker|   sex|tip_percentages|
+------+------+---------------+
|    No|Female|          15.69|
|    No|  Male|           16.1|
|   Yes|  Male|          15.28|
|   Yes|Female|          18.24|
+------+------+---------------+



In [158]:
# Now putting it all together using mean:
tips.groupBy('smoker', 'sex').agg(round(mean(tips.tip_percentage), 2).alias("tip_percentages")).show()

+------+------+---------------+
|smoker|   sex|tip_percentages|
+------+------+---------------+
|    No|Female|          15.69|
|    No|  Male|           16.1|
|   Yes|  Male|          15.28|
|   Yes|Female|          18.24|
+------+------+---------------+



In [159]:
# Using pivot to make the table a little cleaner:

tips.groupBy('sex').pivot('smoker').agg(round(mean(tips.tip_percentage), 2).alias("tip_percentages")).show()

+------+-----+-----+
|   sex|   No|  Yes|
+------+-----+-----+
|Female|15.69|18.24|
|  Male| 16.1|15.28|
+------+-----+-----+



### Done.

### 4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

- Convert the temperatures to farenheight.
- Which month has the most rain, on average?
- Which year was the windiest?
- What is the most frequent type of weather in January?
- What is the average high and low temperature on sunny days in July in 2013 and 2014?
- What percentage of days were rainy in q3 of 2015?
- For each year, find what percentage of days it rained (had non-zero precipitation).

In [117]:
from pyspark.sql.functions import month, year, quarter

In [87]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(6)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



#### Convert the temperatures to farenheight.

In [111]:
# Exocortex source: https://www.programiz.com/python-programming/examples/celsius-fahrenheit

In [115]:
weather_adjust = weather.withColumn("temp_max", expr("ROUND(temp_max * 1.8 + 32)"))
weather_adjust = weather.withColumn("temp_min", expr("ROUND(temp_min * 1.8 + 32)"))
weather_adjust.show()

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|    41.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|    37.0| 4.5|   rain|
|2012-01-03|          0.8|    11.7|    45.0| 2.3|   rain|
|2012-01-04|         20.3|    12.2|    42.0| 4.7|   rain|
|2012-01-05|          1.3|     8.9|    37.0| 6.1|   rain|
|2012-01-06|          2.5|     4.4|    36.0| 2.2|   rain|
|2012-01-07|          0.0|     7.2|    37.0| 2.3|   rain|
|2012-01-08|          0.0|    10.0|    37.0| 2.0|    sun|
|2012-01-09|          4.3|     9.4|    41.0| 3.4|   rain|
|2012-01-10|          1.0|     6.1|    33.0| 3.4|   rain|
|2012-01-11|          0.0|     6.1|    30.0| 5.1|    sun|
|2012-01-12|          0.0|     6.1|    29.0| 1.9|    sun|
|2012-01-13|          0.0|     5.0|    27.0| 1.3|    sun|
|2012-01-14|          4.1|     4.4|    33.0| 5.3|   snow|
|2012-01-15|  

#### Which month has the most rain, on average?

In [162]:
weather.withColumn("month", month("date")).groupBy("month").agg(round(mean("precipitation"), 2).alias("avg_rainfall")).sort(col('avg_rainfall').desc()).show()

+-----+------------+
|month|avg_rainfall|
+-----+------------+
|   11|        5.35|
|   12|        5.02|
|    3|        4.89|
|   10|        4.06|
|    1|        3.76|
|    2|        3.73|
|    4|        3.13|
|    9|        1.96|
|    5|        1.67|
|    8|        1.32|
|    6|        1.11|
|    7|        0.39|
+-----+------------+



#### Which year was the windiest?

- Expecting to have the same process, but instead now I'm looking for the `max` of the wind measurement per year.

In [131]:
weather.show(5)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 5 rows



In [163]:
weather_adjust.withColumn("year", year("date")).groupBy("year").agg(round(max("wind"), 2).alias("max_annual_wind")).sort(col('max_annual_wind').desc()).show()

+----+---------------+
|year|max_annual_wind|
+----+---------------+
|2012|            9.5|
|2013|            8.8|
|2014|            8.8|
|2015|            8.0|
+----+---------------+



#### What is the most frequent type of weather in January?

In [168]:
weather.filter(month("date") == 1).groupBy("weather").agg(count("weather").alias("count")).sort(col("count").desc()).show()

# or instead of .show() I could use .first()

+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
|   rain|   35|
|    sun|   33|
|drizzle|   10|
|   snow|    8|
+-------+-----+



It appears that fog was the most common kind of weather in January.

#### What is the average high and low temperature on sunny days in July in 2013 and 2014?

- So with this problem I'm going to have to filter by the month, and then isloate only the sunny days in each of those months.

In [142]:
(
    weather.filter(month("date") == 7)
    .filter(year("date") > 2012)
    .filter(year("date") < 2015)
    .filter(col("weather") == lit("sun"))
    .agg(
        avg("temp_max").alias("average_high_temp"),
        avg("temp_min").alias("average_low_temp"),
    )
    .show()
)

+------------------+-----------------+
| average_high_temp| average_low_temp|
+------------------+-----------------+
|26.828846153846158|14.18269230769231|
+------------------+-----------------+



#### What percentage of days were rainy in q3 of 2015?

In [143]:
(
    weather.filter(year("date") == 2015)
    .filter(quarter("date") == 3)
    .select(when(col("weather") == "rain", 1).otherwise(0).alias("rain"))
    .agg(mean("rain"))
    .show()
)

+--------------------+
|           avg(rain)|
+--------------------+
|0.021739130434782608|
+--------------------+



Nice, so we used the when/otherwise functions to create a boolean mask.

#### For each year, find what percentage of days it rained (had non-zero precipitation).

In [156]:
weather.withColumn("year", year("date")).select("year", when(col("precipitation") > 0, 1).otherwise(0).alias("rain")).groupBy("year").agg(round(mean('rain'), 2).alias("avg_precip")).show()

+----+----------+
|year|avg_precip|
+----+----------+
|2015|      0.39|
|2013|      0.42|
|2014|      0.41|
|2012|      0.48|
+----+----------+



#### Scratch area

In [116]:
# first step is to create an average temp column:

weather_adjust = weather_adjust.withColumn('temp_avg', expr("ROUND(temp_min + temp_max) / 2"))
weather_adjust.show()

+----------+-------------+--------+--------+----+-------+--------+
|      date|precipitation|temp_max|temp_min|wind|weather|temp_avg|
+----------+-------------+--------+--------+----+-------+--------+
|2012-01-01|          0.0|    12.8|    41.0| 4.7|drizzle|    27.0|
|2012-01-02|         10.9|    10.6|    37.0| 4.5|   rain|    24.0|
|2012-01-03|          0.8|    11.7|    45.0| 2.3|   rain|    28.5|
|2012-01-04|         20.3|    12.2|    42.0| 4.7|   rain|    27.0|
|2012-01-05|          1.3|     8.9|    37.0| 6.1|   rain|    23.0|
|2012-01-06|          2.5|     4.4|    36.0| 2.2|   rain|    20.0|
|2012-01-07|          0.0|     7.2|    37.0| 2.3|   rain|    22.0|
|2012-01-08|          0.0|    10.0|    37.0| 2.0|    sun|    23.5|
|2012-01-09|          4.3|     9.4|    41.0| 3.4|   rain|    25.0|
|2012-01-10|          1.0|     6.1|    33.0| 3.4|   rain|    19.5|
|2012-01-11|          0.0|     6.1|    30.0| 5.1|    sun|    18.0|
|2012-01-12|          0.0|     6.1|    29.0| 1.9|    sun|    1

In [129]:
weather_adjust.withColumn("month", month("date")).groupBy("month").agg(round(mean("temp_avg"), 2).alias("avg_month_temp")).sort('avg_month_temp').show()

+-----+--------------+
|month|avg_month_temp|
+-----+--------------+
|    1|         22.52|
|   12|         23.08|
|    2|         24.58|
|   11|         25.71|
|    3|         26.58|
|    4|         29.24|
|   10|         32.61|
|    5|          34.3|
|    9|         38.08|
|    6|         38.23|
|    7|         41.78|
|    8|         42.35|
+-----+--------------+



In [20]:
mpg.select(sum(mpg.hwy)/count(mpg.hwy).alias("avg_1"), avg(mpg.hwy).alias("avg_2")).show()

+----------------------------------+-----------------+
|(sum(hwy) / count(hwy) AS `avg_1`)|            avg_2|
+----------------------------------+-----------------+
|                 23.44017094017094|23.44017094017094|
+----------------------------------+-----------------+



In [23]:
mpg.select(concat(mpg.manufacturer, lit(" "), lit(mpg.model))).show(5)

+------------------------------+
|concat(manufacturer,  , model)|
+------------------------------+
|                       audi a4|
|                       audi a4|
|                       audi a4|
|                       audi a4|
|                       audi a4|
+------------------------------+
only showing top 5 rows



In [21]:
mpg.select(sum(mpg.hwy) / count(mpg.hwy).alias("average_1")).show()

+--------------------------------------+
|(sum(hwy) / count(hwy) AS `average_1`)|
+--------------------------------------+
|                     23.44017094017094|
+--------------------------------------+



In [24]:
mpg.select(sum(mpg.hwy)).show()

+--------+
|sum(hwy)|
+--------+
|    5485|
+--------+

